In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score
import xgboost as xgb

In [2]:
df = pd.read_csv("CKD.csv",index_col=None)

In [3]:
df = pd.get_dummies(df, drop_first=True)

In [4]:
df

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [5]:
X = df.drop("classification_yes", axis=1)
y = df["classification_yes"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [6]:

rf = RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=0)
rf.fit(X_train, y_train)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)

C:\Anaconda\envs\mlai\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [7]:

# 1. Random Forest Feature Importances
rf_importance = pd.Series(rf.feature_importances_, index=X.columns, name="RandomForest")

# 2. Logistic Regression Coefficients (absolute)
coefficients = logreg.coef_[0]
logreg_importance = pd.Series(coefficients, index=X.columns, name="LogisticRegression")

# 3. Permutation Importance
perm = permutation_importance(rf, X_test, y_test, n_repeats=10, random_state=42)
perm_importance = pd.Series(perm.importances_mean, index=X.columns, name="Permutation")

# 4. XGBoost Feature Importances
xgb_importance = pd.Series(xgb_model.feature_importances_, index=X.columns, name="XGBoost")

In [8]:
all_importances = pd.concat([rf_importance, logreg_importance, perm_importance, xgb_importance], axis=1)

In [9]:
print(all_importances.round(4))

             RandomForest  LogisticRegression  Permutation  XGBoost
age                0.0202             -0.0223       -0.006   0.0083
bp                 0.0042              0.0768       -0.002   0.0055
al                 0.0720              0.1302        0.002   0.1235
su                 0.0075              0.0091       -0.003   0.0000
bgr                0.0337              0.0430       -0.003   0.0470
bu                 0.0489              0.0102       -0.002   0.0053
sc                 0.0459              0.1280       -0.007   0.0108
sod                0.0324              0.0805        0.000   0.0032
pot                0.0134             -0.0013        0.000   0.0041
hrmo               0.2011             -0.2130        0.026   0.3231
pcv                0.3116             -0.4771        0.004   0.1574
wc                 0.0081              0.0002        0.000   0.0048
rc                 0.1182             -0.0483        0.000   0.0078
sg_b               0.0085             -0.0398   